In [1]:
from ipynb.fs.full.A_matrix import *

40
5
40
Shear Modulus (mu): 76923076923.07692
Lame Modulus (Lambda): 115384615384.61539


In [2]:
class boundaryCellDisplacement(A):

    def __init__(self, edges, xy):
        
        sw_S, sw_N, sw_W, sw_E = 1, 1, 1, 1

        for edge in edges:
            if edge == "b": sw_S = 2
            if edge == "t": sw_N = 2
            if edge == "l": sw_W = 2
            if edge == "r": sw_E = 2

        if transient:
            self.a_P = (rho*dx*dy/(dt**2)) + A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W
        else:
            self.a_P = A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W

        self.a_N = A(xy).a_N*sw_N
        self.a_S = A(xy).a_S*sw_S
        self.a_E = A(xy).a_E*sw_E
        self.a_W = A(xy).a_W*sw_W

    def b_temp(U_old, U_old_old, k, xy):      
        return A.b_temp(U_old, U_old_old, k, xy)
    
    def b_diff(edges, k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                    (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dy)
                ) 

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion
    
    def corner(edges, corner_placement, uv, U_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, U_previous, uv_i)

        if corner_placement == "NE":
            corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

        for edge in edges:
            if (edge == "b") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.S)
            if (edge == "b") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.S)

            if (edge == "t") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.N)
            if (edge == "t") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.N)

            if (edge == "l") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.W)
            if (edge == "l") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.W)

            if (edge == "r") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.E)
            if (edge == "r") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.E)

        if len(edges) > 1:

            # The order of edges shouldn't matter shouldn't matter
            for edge in edges:
                if edge == "b":
                    for edge in edges:
                        if (edge == "l") & (corner_placement == "SW"):
                            corner = disp.SW 
                        if (edges == "r") & (corner_placement == "SE"):
                            corner = disp.SE
                if edge == "t":
                    for edge in edges:
                        if (edge == "l") & (corner_placement == "NW"):
                            corner = disp.NW
                        if (edges == "r") & (corner_placement == "NE"):
                            corner = disp.NE



            # if (edges[0] == "b") & (edges[1] == "l")& (corner_placement == "SW"):
            #     corner = disp.SW 
            # if (edges[0] == "b") & (edges[1] == "r") & (corner_placement == "SE"):
            #     corner = disp.SE
            # if (edges[0] == "t") & (edges[1] == "l") & (corner_placement == "NW"):
            #     corner = disp.NW
            # if (edges[0] == "t") & (edges[1] == "r") & (corner_placement == "NE"):
            #     corner = disp.NE

        
        return corner

# boundaryCellDisplacement.b_diff(["b", "l"], 14, "x", U_previous)




In [ ]:
def displacement_cell_BCs(A_matrix, b_matrix, k, edges, xy, U_old, U_old_old, U_previous):

    A_matrix[k,k] = boundaryCellDisplacement(edges, xy).a_P
    #an
    A_matrix[k,index(k).n] = - boundaryCellDisplacement(edges, xy).a_N        
    #as
    A_matrix[k,index(k).s] = - boundaryCellDisplacement(edges, xy).a_S   
    #ae
    A_matrix[k, index(k).e] = - boundaryCellDisplacement(edges, xy).a_E
    #aw
    A_matrix[k, index(k).w] = - boundaryCellDisplacement(edges, xy).a_W

    b_matrix[k] =(
                boundaryCellDisplacement.b_temp(U_old, U_old_old, k, xy)
                +
                boundaryCellDisplacement.b_diff(edges, k, xy, U_previous)
            )   

    return A_matrix, b_matrix

def displacement_point_BCs(A_matrix, b_matrix, k, edges, xy):
    
    A_matrix[k,k] = 1e10
    b_matrix[k] = edge_U(edges, xy).BC*1e10

    return A_matrix, b_matrix